## Timeseries Modeling - Lasso_cvglmnet
### Objective:
The objective of the notebook is to -
* Build the final model of Lasso_cvglmnet algorithm and score the test set to get a performance metric
* For forecasting future periods, we will re-train the model on the train + validation + test set to capture the patterns in the test set and then forecast future N periods

In [0]:
import pandas as  pd
import numpy as np
import datetime as dt
import os
import logging
import scipy
from datetime import datetime
import sys
import joblib
import multiprocessing
import random
import mlflow
import dotsi
from sklearn.metrics import mean_absolute_error,mean_squared_error

import glmnet_python
from cvglmnet import cvglmnet
from glmnet_python.cvglmnetCoef import cvglmnetCoef

from glmnetSet import glmnetSet
from glmnetPredict import glmnetPredict
from glmnet import glmnet
from cvelnet import cvelnet
from cvlognet import cvlognet
from cvmultnet import cvmultnet
from cvmrelnet import cvmrelnet
from cvfishnet import cvfishnet

In [0]:
# logging part
p_dir = "/tmp/"
log_file = "lasso_cvglmnet_model" + " (" +datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+ ").log"

# Lasso_cvglmnet - Hyperparameter tuning logs
logger = logging.getLogger('custom_log')
logger.setLevel(logging.DEBUG)

# Applying necessary formatter
fh = logging.FileHandler(p_dir+log_file)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
fh.setFormatter(formatter)
logger.addHandler(fh)

#### Processing Config file
Dependent variable, date variable, modeling granularity & other related modeling details are provided in the form of a config file. Each TS Algorithm, and the related hyperparameter values to be tried, should be given in the config file.

In [0]:
%run ../../0_Config.ipynb

In [0]:
logger.info("Config file read")

# For exporting the config file
temp_config = app_config.copy()

In [0]:
# Create the algo directory for storing the results
output_directory = app_config['output_dir_path']
root_dir = "Modeling_Results"
algorithm = "Lasso_cvglmnet"
algo_path = os.path.join(output_directory,root_dir,algorithm)
if not os.path.exists(algo_path):
    os.makedirs(algo_path)
logger.info("Created algorithm directory")    

logs_path = os.path.join(output_directory,root_dir,'logs',algorithm)
if not os.path.exists(logs_path):
    os.makedirs(logs_path)
logger.info("Created logs directory")

config_path = os.path.join(app_config['output_dir_path'],"Modeling_Results","config")
if not os.path.exists(config_path):
    os.makedirs(config_path)
logger.info("Created config directory")

#### Broadcasting the required variables
Variables suffixed with "_conf" are taken from the config file

In [0]:
modeling_granularity_conf = app_config["modeling_granularity"]
# print(modeling_granularity_conf)

# Rename Start date and DV config
dv_config = app_config["dependent_variable"]
ds_config = app_config["date_var"]

# pos and neg corr broadcast
corr_config = dict(app_config['Algorithms']['Lasso_cvglmnet']['exogenous_variables'])
corr_config_broadcast = dotsi.Dict({"value":corr_config})

# Limits and penalty
req_info = app_config['Algorithms']['Lasso_cvglmnet']['limits'] 
req_info['zero_penalty_vars'] = app_config['Algorithms']['Lasso_cvglmnet']['zero_penalty_vars']
for key in req_info['coeff_bounds'].keys():
    res = req_info['coeff_bounds'][key]
    res = [-np.inf if x=='-inf' else x for x in res]
    res = [np.inf if x=='inf' else x for x in res]
    req_info['coeff_bounds'][key] = res
broadcast_req_info = dotsi.Dict({"value":req_info})

broadcast_test_periods = dotsi.Dict({"value":app_config["validation"]["no_of_test_periods"]})
broadcast_regressors = dotsi.Dict({"value":list(set(corr_config['positive_corr']+corr_config['negative_corr']+corr_config['uncertain_corr']))})
broadcast_granularity = dotsi.Dict({"value":modeling_granularity_conf})
broadcast_agg_metrics_req = dotsi.Dict({"value":app_config["validation"]["agg_metrics_req"]})
broadcast_tracking = dotsi.Dict({"value":app_config['tracking']})
broadcast_coeff_req = dotsi.Dict({"value":app_config['Algorithms']['Lasso_cvglmnet']['coefficients_required']})
mlflow_tracking_check = dotsi.Dict({"value":"Out of Sample"})
logger.info("Broadcasted the required variables")

In [0]:
# Reading feature selected output and using the significant variables as idvs in modeling
feature_selection_info = app_config['Algorithms']['Lasso_cvglmnet']['feature_selection']
broadcast_use_features = dotsi.Dict({"value":feature_selection_info['use_feature_selected_idvs']})
if(feature_selection_info['use_feature_selected_idvs']):
    if(feature_selection_info['approach']=='lasso_cvglmnet'):
        output_folder = app_config['output_dir_path']+"/Feature_Selection/Lasso/"
    # Reading the latest input file based on timestamp
    coeff_op_files = [file for file in os.listdir(output_folder)]
    coeff_op_files = [file.replace(".csv","") for file in coeff_op_files]
    version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in coeff_op_files]
    max_date = max(version_dates)
    max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
    req_file_name = [x for x in coeff_op_files if max_date in x]
    coeff_op_file_path = os.path.join(output_folder,req_file_name[0]+".csv")
    print(coeff_op_file_path)

    # Reading the data
    coeff_df = pd.read_csv(coeff_op_file_path)
    coeff_df = coeff_df[coeff_df['status']=='success']
    # print(coeff_df.shape)
    coeff_df[modeling_granularity_conf] = coeff_df[modeling_granularity_conf].astype(str)
    idvs_len = len(feature_selection_info['must_have_idvs'])
    if(idvs_len>0):
        temp1 = coeff_df[modeling_granularity_conf].drop_duplicates()
        temp1['temp'] = 1
        temp2 = pd.DataFrame({'IDV':feature_selection_info['must_have_idvs']})
        temp2['temp'] = 1
        temp = temp1.join(temp2, on = 'temp', how ='left')
        req_cols = modeling_granularity_conf + ['IDV']
        coeff_df = coeff_df.drop_duplicates()
    coeffs_broadcast = dotsi.Dict({"value":coeff_df})
# display(coeff_df)

/dbfs/mnt/solutionsadls_data/modeling_results_python/Feature_Selection/Lasso/lasso_feature_selection_results (2023-02-08-11-27-11).csv

In [0]:
# -*- coding: utf-8 -*-
"""
--------------------------------------------------------------------------
 cvglmnet.m: cross-validation for glmnet
--------------------------------------------------------------------------

 DESCRIPTION:
    Does k-fold cross-validation for glmnet, produces a plot, and returns
    a value for lambdau. Cross-validation is not implemented for Cox model yet.

 USAGE:

    Note that like glmnet, all arguments are keyword-only:
 
    CVerr = cvglmnet(x, y, family, options, type, nfolds, foldid,
    parallel, keep, grouped);

    Fewer input arguments(more often) are allowed in the call. Default values
    for the arguments are used unless specified by the user.
        
=======================
INPUT ARGUMENTS
 x           nobs x nvar scipy 2D array of x parameters (as in glmnet).
 y           nobs x nc scipy Response y as in glmnet.
 family      Response type as family in glmnet.
 options     Options as in glmnet.
 ptype       loss to use for cross-validation. Currently five options, not
             all available for all models. The default is ptype='deviance', which uses
             squared-error for Gaussian models (a.k.a ptype='mse' there), deviance for
             logistic and Poisson regression, and partial-likelihood for the Cox
             model (Note that CV for cox model is not implemented yet). 
             ptype='class' applies to binomial and multinomial logistic
             regression only, and gives misclassification error. ptype='auc' is for
             two-class logistic regression only, and gives area under the ROC curve.
             ptype='mse' or ptype='mae' (mean absolute error) can be used by all models
             except the 'cox'; they measure the deviation from the fitted mean to the
             response.  
 nfolds      number of folds - default is 10. Although nfolds can be as
             large as the sample size (leave-one-out CV), it is not recommended for
             large datasets. Smallest value allowable is nfolds=3.
 foldid      an optional vector of values between 1 and nfold identifying
             what fold each observation is in. If supplied, nfold can be
             missing.
 parallel    If True, use parallel computation to fit each fold. 
 keep        If keep=True, a prevalidated array is returned containing
             fitted values for each observation and each value of lambda.
             This means these fits are computed with this observation and
             the rest of its fold omitted. The foldid vector is also
             returned. Default is keep=False.   
 grouped     This is an experimental argument, with default true, and can
             be ignored by most users. For all models except the 'cox',
             this refers to computing nfolds separate statistics, and then
             using their mean and estimated standard error to describe the
             CV curve. If grouped=false, an error matrix is built up at
             the observation level from the predictions from the nfold
             fits, and then summarized (does not apply to
             type='auc'). For the 'cox' family, grouped=true obtains the 
             CV partial likelihood for the Kth fold by subtraction; by
             subtracting the log partial likelihood evaluated on the full
             dataset from that evaluated on the on the (K-1)/K dataset.
             This makes more efficient use of risk sets. With
             grouped=FALSE the log partial likelihood is computed only on
             the Kth fold.

=======================
OUTPUT ARGUMENTS:
 A dict() is returned with the following fields.
 lambdau     the values of lambda used in the fits.
 cvm         the mean cross-validated error - a vector of length
             length(lambdau). 
 cvsd        estimate of standard error of cvm.
 cvup        upper curve = cvm+cvsd.
 cvlo        lower curve = cvm-cvsd.
 nzero       number of non-zero coefficients at each lambda.
 name        a text string indicating type of measure (for plotting
             purposes). 
 glmnet_fit  a fitted glmnet object for the full data.
 lambda_min  value of lambda that gives minimum cvm.
 lambda_1se  largest value of lambda such that error is within 1 standard
             error of the minimum. 
 class       Type of regression - internal usage.
 fit_preval  if keep=true, this is the array of prevalidated fits. Some
             entries can be NA, if that and subsequent values of lambda
             are not reached for that fold.
 foldid      if keep=true, the fold assignments used.

 DETAILS:
    The function runs glmnet nfolds+1 times; the first to get the lambda
    sequence, and then the remainder to compute the fit with each of the 
    folds omitted. The error is accumulated, and the average error and 
    standard deviation over the folds is computed. Note that cvglmnet 
    does NOT search for values for alpha. A specific value should be 
    supplied, else alpha=1 is assumed by default. If users would like to 
    cross-validate alpha as well, they should call cvglmnet with a 
    pre-computed vector foldid, and then use this same fold vector in 
    separate calls to cvglmnet with different values of alpha. 

 LICENSE: GPL-2

 AUTHORS:
    Algorithm was designed by Jerome Friedman, Trevor Hastie and Rob Tibshirani
    Fortran code was written by Jerome Friedman
    R wrapper (from which the MATLAB wrapper was adapted) was written by Trevor Hasite
    The original MATLAB wrapper was written by Hui Jiang,
    and is updated and maintained by Junyang Qian.
    This Python wrapper (adapted from the Matlab and R wrappers) is written by Balakumar B.J., 
    Department of Statistics, Stanford University, Stanford, California, USA.

 REFERENCES:
    Friedman, J., Hastie, T. and Tibshirani, R. (2008) Regularization Paths for Generalized Linear Models via Coordinate Descent, 
    http://www.jstatsoft.org/v33/i01/
    Journal of Statistical Software, Vol. 33(1), 1-22 Feb 2010
    
    Simon, N., Friedman, J., Hastie, T., Tibshirani, R. (2011) Regularization Paths for Cox's Proportional Hazards Model via Coordinate Descent,
    http://www.jstatsoft.org/v39/i05/
    Journal of Statistical Software, Vol. 39(5) 1-13

    Tibshirani, Robert., Bien, J., Friedman, J.,Hastie, T.,Simon, N.,Taylor, J. and Tibshirani, Ryan. (2010) Strong Rules for Discarding Predictors in Lasso-type Problems,
    http://www-stat.stanford.edu/~tibs/ftp/strong.pdf
    Stanford Statistics Technical Report

 SEE ALSO:
    cvglmnetPlot, cvglmnetCoef, cvglmnetPredict, and glmnet.

 EXAMPLES:
 
      # Gaussian
      x = scipy.random.rand(100, 10)
      y = scipy.random.rand(100, 1)
      cvfit = cvglmnet(x = x, y = y)
      cvglmnetPlot(cvfit)
      print( cvglmnetCoef(cvfit) )
      print( cvglmnetPredict(cvfit, x[0:5, :], 'lambda_min') )
      cvfit1 = cvglmnet(x = x, y = y, ptype = 'mae')
      cvglmnetPlot(cvfit1)
      
      # Binomial
      x = scipy.random.rand(100, 10)
      y = scipy.random.rand(100,1)
      y = (y > 0.5)*1.0
      fit = cvglmnet(x = x, y = y, family = 'binomial', ptype = 'class')    
      cvglmnetPlot(fit)
      
      # poisson
      x = scipy.random.rand(100,10)
      y = scipy.random.poisson(size = [100, 1])*1.0
      cvfit = cvglmnet(x = x, y = y, family = 'poisson')
      cvglmnetPlot(cvfit)
      
      # Multivariate Gaussian:
      x = scipy.random.rand(100, 10)
      y = scipy.random.rand(100,3)
      cvfit = cvglmnet(x = x, y = y, family = 'mgaussian')      
      cvglmnetPlot(cvfit)
       
      # Multinomial
      x = scipy.random.rand(100,10)
      y = scipy.random.rand(100,1)
      y[y < 0.3] = 1.0
      y[y < 0.6] = 2.0
      y[y < 1.0] = 3.0
      cvfit = cvglmnet(x = x, y = y, family = 'multinomial')
      cvglmnetPlot(cvfit) 
      
      #cox
      Not implemented for cvglmnet.py


    
 % Cox
    n=1000;p=30;
    nzc=p/3;
    x=randn(n,p);
    beta=randn(nzc,1);
    fx=x(:,1:nzc)*beta/3;
    hx=exp(fx);
    ty=exprnd(1./hx,n,1);
    tcens=binornd(1,0.3,n,1);
    y=cat(2,ty,1-tcens);
    foldid=randsample(10,n,true);
    fit1_cv=cvglmnet(x,y,'cox',[],[],[],foldid);
    cvglmnetPlot(fit1_cv);
    
 % Parallel
    matlabpool;
    x=randn(1e3,100);
    y=randn(1e3,1);
    tic;
    cvglmnet(x,y);
    toc;
    tic;
    cvglmnet(x,y,[],[],[],[],[],true);
    toc;

"""


def cvglmnet(*, x,
             y,
             family = 'gaussian',
             ptype = 'default',
             nfolds = 10,
             foldid = scipy.empty([0]),
             parallel = False,
             keep = False,
             grouped = True,
             **options):

    options = glmnetSet(options)

    if 0 < len(options['lambdau']) < 2:
        raise ValueError('Need more than one value of lambda for cv.glmnet')
    
    nobs = x.shape[0]

    # we should not really need this. user must supply the right shape
    # if y.shape[0] != nobs:
    #    y = scipy.transpose(y)
        
    # convert 1d python array of size nobs to 2d python array of size nobs x 1
    if len(y.shape) == 1:
        y = scipy.reshape(y, [y.size, 1])

    # we should not really need this. user must supply the right shape       
    # if (len(options['offset']) > 0) and (options['offset'].shape[0] != nobs):
    #    options['offset'] = scipy.transpose(options['offset'])
    
    if len(options['weights']) == 0:
        options['weights'] = scipy.ones([nobs, 1], dtype = scipy.float64)

    # main call to glmnet        
    glmfit = glmnet(x = x, y = y, family = family, **options)    

    is_offset = glmfit['offset']
    options['lambdau'] = glmfit['lambdau']
    
    nz = glmnetPredict(glmfit, scipy.empty([0]), scipy.empty([0]), 'nonzero')
    if glmfit['class'] == 'multnet':        
        nnz = scipy.zeros([len(options['lambdau']), len(nz)])
        for i in range(len(nz)):
            nnz[:, i] = scipy.transpose(scipy.sum(nz[i], axis = 0))
        nz = scipy.ceil(scipy.median(nnz, axis = 1))    
    elif glmfit['class'] == 'mrelnet':
        nz = scipy.transpose(scipy.sum(nz[0], axis = 0))
    else:
        nz = scipy.transpose(scipy.sum(nz, axis = 0))
    
    if len(foldid) == 0:
        ma = scipy.tile(scipy.arange(nfolds), [1, int(scipy.floor(nobs/nfolds))])
        mb = scipy.arange(scipy.mod(nobs, nfolds))
        mb = scipy.reshape(mb, [1, mb.size])
        population = scipy.append(ma, mb, axis = 1)
        mc = scipy.random.permutation(len(population))
        mc = mc[0:nobs]
        foldid = population[mc]
        foldid = scipy.reshape(foldid, [foldid.size,])
    else:
        nfolds = scipy.amax(foldid) + 1
        
    if nfolds < 3:
        raise ValueError('nfolds must be bigger than 3; nfolds = 10 recommended')        
        
    cpredmat = list()
    foldid = scipy.reshape(foldid, [foldid.size, ])
    if parallel == True:
        num_cores = multiprocessing.cpu_count()
        sys.stderr.write("[status]\tParallel glmnet cv with " + str(num_cores) + " cores\n")
        cpredmat = joblib.Parallel(n_jobs=num_cores)(joblib.delayed(doCV)(i, x, y, family, foldid, nfolds, is_offset, **options) for i in range(nfolds))
    else:
        for i in range(nfolds):
            newFit = doCV(i, x, y, family, foldid, nfolds, is_offset, **options)
            cpredmat.append(newFit)
        
    if cpredmat[0]['class'] == 'elnet':
        cvstuff = cvelnet( cpredmat, options['lambdau'], x, y \
                          , options['weights'], options['offset'] \
                          , foldid, ptype, grouped, keep)
    elif cpredmat[0]['class'] == 'lognet':
        cvstuff = cvlognet(cpredmat, options['lambdau'], x, y \
                          , options['weights'], options['offset'] \
                          , foldid, ptype, grouped, keep)
    elif cpredmat[0]['class'] == 'multnet':
        cvstuff = cvmultnet(cpredmat, options['lambdau'], x, y \
                          , options['weights'], options['offset'] \
                          , foldid, ptype, grouped, keep)
    elif cpredmat[0]['class'] == 'mrelnet':
        cvstuff = cvmrelnet(cpredmat, options['lambdau'], x, y \
                          , options['weights'], options['offset'] \
                          , foldid, ptype, grouped, keep)
    elif cpredmat[0]['class'] == 'fishnet':
        cvstuff = cvfishnet(cpredmat, options['lambdau'], x, y \
                           , options['weights'], options['offset'] \
                           , foldid, ptype, grouped, keep)
    elif cpredmat[0]['class'] == 'coxnet':
        raise NotImplementedError('Cross-validation for coxnet not implemented yet.')
        #cvstuff = cvcoxnet(cpredmat, options['lambdau'], x, y \
        #                  , options['weights'], options['offset'] \
        #                  , foldid, ptype, grouped, keep)
 
    cvm = cvstuff['cvm']
    cvsd = cvstuff['cvsd']
    cvname = cvstuff['name']

    CVerr = dict()
    CVerr['lambdau'] = options['lambdau']       
    CVerr['cvm'] = scipy.transpose(cvm)
    CVerr['cvsd'] = scipy.transpose(cvsd)
    CVerr['cvup'] = scipy.transpose(cvm + cvsd)
    CVerr['cvlo'] = scipy.transpose(cvm - cvsd)
    CVerr['nzero'] = nz
    CVerr['name'] = cvname
    CVerr['glmnet_fit'] = glmfit
    if keep:
        CVerr['fit_preval'] = cvstuff['fit_preval']
        CVerr['foldid'] = foldid
    if ptype == 'auc':
        cvm = -cvm
    CVerr['lambda_min'] = scipy.amax(options['lambdau'][cvm <= scipy.amin(cvm)]).reshape([1])  
    idmin = options['lambdau'] == CVerr['lambda_min']
    semin = cvm[idmin] + cvsd[idmin]
    CVerr['lambda_1se'] = scipy.amax(options['lambdau'][cvm <= semin]).reshape([1])
    CVerr['class'] = 'cvglmnet'
    
    return(CVerr)
        
# end of cvglmnet
#==========================
def doCV(i, x, y, family, foldid, nfolds, is_offset, **options):
    which = foldid == i
    opts = options.copy()
    opts['weights'] = opts['weights'][~which, ]
    opts['lambdau'] = options['lambdau']
    if is_offset:
        if opts['offset'].size > 0:
            opts['offset'] = opts['offset'][~which, ]
    xr = x[~which, ]
    yr = y[~which, ]
    newFit = glmnet(x = xr, y = yr, family = family, **opts)    
    return(newFit)
  
###Prediction function
def cvglmnetPredict(obj, newx = None, s = 'lambda_1se', **options):
    if newx is None:
        CVpred = cvglmnetCoef(obj)
        return(CVpred)
        
    if type(s) == scipy.ndarray and s.dtype == 'float64':
        lambdau = s
    elif s in ['lambda_1se', 'lambda_min']:
            lambdau = obj[s]
    else:
            raise ValueError('Invalid form for s')
    
    CVpred = glmnetPredict(obj['glmnet_fit'], newx, lambdau, **options)
    
    return(CVpred)

<command-3460216454770823>:203: DeprecationWarning: scipy.empty is deprecated and will be removed in SciPy 2.0.0, use numpy.empty instead
 foldid = scipy.empty([0]),

In [0]:
# Contribution calculation using the coefficients
def contribution_usingCoeffs(df,coeffs_df,broadcast_gran):
    
    cont_df = df[broadcast_gran+['ds','y','yhat','test_flag','test_flag_agg']+list(set(coeffs_df['Variable']) - {'Intercept'})].reset_index(drop = True)

    for var in coeffs_df['Variable']:
        if(var != 'Intercept'):
            cont_df[var] = cont_df[var]*coeffs_df[coeffs_df['Variable']==var]['Value'].values[0]
        else:
            cont_df[var] = coeffs_df[coeffs_df['Variable']==var]['Value'].values[0]
    return cont_df

In [0]:
def get_forecast_UDF(df_data: pd.DataFrame)-> pd.DataFrame:
    """Function to perform model building using the entire data to get the significant features utilizing the broadcasted details from the config file

    Parameters
    ----------
    df_data : pd.DataFrame
        The dataset containing values for all the required variables

    Returns
    -------
    pd.DataFrame
        Returns a dataframe with the granularity x variable
    """
    try:
        test_periods = int(broadcast_test_periods.value)
        if(broadcast_agg_metrics_req.value == True):
            train_index_start = df_data["train_index_start"].iloc[0]
            train_index_end = df_data["train_index_end"].iloc[0]
            test_i = df_data["test_index_end"].iloc[0]
            window_no = str(str(train_index_start)+" "+str(train_index_end)+" "+str(test_i)+" "+str(df_data["window_no"].iloc[0]))
        else:
            train_index_end = len(df_data) - test_periods
            test_i = len(df_data)
            window_no = str(1)
            
        df_data = df_data.sort_values(by=['ds'],ascending=True).reset_index(drop = True)

        # broadcast_granularity
        broadcast_gran = broadcast_granularity.value
        
        # Appending regressors based on the sign of correlation
        corr_var = corr_config_broadcast.value
        
        # Train - test split
        train = df_data.iloc[:train_index_end]
        test = df_data.iloc[train_index_end:test_i]
        
        req_info = broadcast_req_info.value
        if(broadcast_use_features.value==True):
            # Reading regressors from feature selection
            coeffs_df = coeffs_broadcast.value
            for x in broadcast_gran:
                coeffs_df = coeffs_df[coeffs_df[x] == df_data[x].iloc[0]]
            regressors = list(coeffs_df['IDV'].values)
            temp_list = list(set(broadcast_regressors.value) - set(regressors))
        else:
            regressors = list(set(corr_var["positive_corr"] + corr_var["negative_corr"]+corr_var['uncertain_corr']))
            temp_list1 = []
            # Removing regressors based on the correlation
            if(corr_var["consider_correlation"]):   
                for x in corr_var["positive_corr"]:
                    if(train[['y',x]].corr().iloc[0][1]<0):
                        temp_list1.append(x)
                for x in corr_var["negative_corr"]:
                    if (x not in temp_list1):
                        if(train[['y',x]].corr().iloc[0][1]>0):
                            temp_list1.append(x)   
                regressors = list(set(regressors) - set(temp_list1))
            
            # Checking for variance in the regressor
            temp_list2 = []
            if len(regressors)>0:
                for ex_var in regressors:  
                    mean = train[ex_var].mean()
                    std = train[ex_var].std()
                    if mean == 0:
                        if std <= 0.001:
                            temp_list2.append(ex_var)
                    else:
                        if abs(std/mean) <= 0.01:
                            temp_list2.append(ex_var)

            regressors = list(set(regressors) - set(temp_list2))
            reg_len = len(regressors)

            #################### Adding lower and upper limits #################
            cl = np.array([np.repeat(-np.inf,reg_len), np.repeat(np.inf,reg_len)], dtype = scipy.float64)
            if(req_info['consider_limits']):
                zero_lower_cols = list(set(regressors) & set(corr_var["positive_corr"]))
                zero_upper_cols = list(set(regressors) & set(corr_var["negative_corr"]))

                for j in zero_lower_cols:
                    if j in regressors:
                        indx = regressors.index(j)
                        cl[0][indx]= 0  

                for j in zero_upper_cols:
                    if j in regressors:
                        indx = regressors.index(j)
                        cl[1][indx]= 0     
            limits_check = (pd.DataFrame({"IDV":regressors,"Lower_limit":cl[0],"Upper_limit":cl[1]})).sort_values(by = "IDV")

            ########################## Adding Penalty factor ########################
            pfac = np.ones([1, reg_len])
            zero_pen_cols = req_info['zero_penalty_vars']
            zero_pen_cols = [var for var in zero_pen_cols if var in regressors]
            if(reg_len <= len(zero_pen_cols)):
                zero_pen_cols = zero_pen_cols[:-1]
            for j in zero_pen_cols:
                indx = regressors.index(j)
                pfac[0][indx]= 0

            penalty_check = (pd.DataFrame({"IDV":regressors,"Penalty":pfac[0]})).sort_values(by = "IDV")

            ##################### Adding weights ######################
            wts = np.ones((len(train),1))

            # Run lasso model 20 times and including only those variables that appear more than 10 times
            seeds = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]
            # Model Function call for the selected seeds
            model_coefficients_check = pd.DataFrame()
            for se in seeds:
                random.seed(se)
                cvfit = cvglmnet(x = train[regressors].to_numpy().copy(), y = train[['y']].astype(float).to_numpy().copy(), \
                                 family = 'gaussian',ptype = 'mse', nfolds = 20,weights=wts,penalty_factor = pfac,cl=cl)
                coef_val = (cvglmnetCoef(cvfit, s = 'lambda_1se'),cvfit['lambda_1se'])[0]
                model_coefficients_2 = pd.DataFrame(data = [["Intercept",coef_val[0]]],columns =["IDV","Value"])
                for i in range(0,reg_len):
                    if(abs(coef_val[i+1])>0):
                        k = pd.DataFrame(data = [[regressors[i],coef_val[i+1]]],columns =["IDV","Value"])
                        model_coefficients_2 = model_coefficients_2.append(k,ignore_index = True)
                model_coefficients_check = model_coefficients_check.append(model_coefficients_2,ignore_index = True)

            model_coefficients_Acting =  model_coefficients_check.groupby('IDV').size().reset_index().rename(columns = {0:"Frequency"})
            model_coefficients_Acting = pd.merge(model_coefficients_Acting,model_coefficients_check,on="IDV",how = "left")
            model_coefficients_Acting =model_coefficients_Acting.drop_duplicates(subset=['IDV'],keep = 'first').reset_index(drop = True)
            model_coefficients_final = model_coefficients_Acting[model_coefficients_Acting['Frequency']>=10].reset_index(drop = True)
            model_coefficients_final = model_coefficients_final[model_coefficients_final['IDV']!= "Intercept"]
            temp_list3 = list(set(regressors) - set(list(model_coefficients_final['IDV'].values)))
            regressors = list(model_coefficients_final['IDV'].values)
            temp_list = temp_list1 + temp_list2 + temp_list3
            
        
        reg_len = len(regressors)
        #################### Adding lower and upper limits #################
        cl = np.array([np.repeat(-np.inf,reg_len), np.repeat(np.inf,reg_len)], dtype = scipy.float64)
        if(req_info['consider_limits']):
            zero_lower_cols = list(set(regressors) & set(corr_var["positive_corr"]))
            zero_upper_cols = list(set(regressors) & set(corr_var["negative_corr"]))
            
            for j in zero_lower_cols:
                if j in regressors:
                    indx = regressors.index(j)
                    cl[0][indx]= 0  

            for j in zero_upper_cols:
                if j in regressors:
                    indx = regressors.index(j)
                    cl[1][indx]= 0     

        limits_check = (pd.DataFrame({"IDV":regressors,"Lower_limit":cl[0],"Upper_limit":cl[1]})).sort_values(by = "IDV")
        for var in req_info['coeff_bounds'].keys():
            val = req_info['coeff_bounds'][var]
            limits_check.loc[limits_check.IDV==var,['Lower_limit','Upper_limit']] = val
            if var in regressors:
                indx = regressors.index(var)
                cl[1][indx]= val[1]  
                cl[0][indx]= val[0] 
        
        ########################## Adding Penalty factor ########################
        pfac = np.ones([1, reg_len])
        zero_pen_cols = req_info['zero_penalty_vars']
        zero_pen_cols = [var for var in zero_pen_cols if var in regressors]
        if(reg_len <= len(zero_pen_cols)):
            zero_pen_cols = zero_pen_cols[:-1]
        for j in zero_pen_cols:
            indx = regressors.index(j)
            pfac[0][indx]= 0

        penalty_check = (pd.DataFrame({"IDV":regressors,"Penalty":pfac[0]})).sort_values(by = "IDV")
        
        ##################### Adding weights ######################
        wts = np.ones((len(train),1))
        
        random.seed(123)
        cvfit = cvglmnet(x = train[regressors].to_numpy().copy(), y = train[['y']].astype(float).to_numpy().copy(),\
                         family = 'gaussian',ptype = 'mse', nfolds = 20,weights=wts,penalty_factor = pfac,cl=cl)
        coef_val = (cvglmnetCoef(cvfit, s = 'lambda_1se'),cvfit['lambda_1se'])[0]
        coeffs_df = pd.DataFrame(data = [["Intercept",coef_val[0]]],columns =["Variable","Value"])
        for i in range(0,reg_len):
            if(abs(coef_val[i+1])>0):
                k = pd.DataFrame(data = [[regressors[i],coef_val[i+1]]],columns =["Variable","Value"])
                coeffs_df = coeffs_df.append(k,ignore_index = True)
        coeffs_df['Value']=coeffs_df['Value'].apply(lambda x: x[0])
        
        test['yhat'] = cvglmnetPredict(cvfit,test[regressors].to_numpy().copy(),s = 'lambda_1se')
        test[['test_flag','test_flag_agg']] = 1
        train['yhat'] = cvglmnetPredict(cvfit,train[regressors].to_numpy().copy(),s = 'lambda_1se')
        train['test_flag'] = np.where(train['ds']<df_data.iloc[-test_periods:]['ds'].min(),0,1)
        train['test_flag_agg'] = 0
        forecast_pd = pd.concat([train,test],ignore_index = True)
        
        # Contribution calculation
        results_pd = contribution_usingCoeffs(forecast_pd,coeffs_df,broadcast_gran)
        # Sales or Quantity can't be negative hence
        results_pd["yhat"] = np.where(results_pd["yhat"]<0,0,results_pd["yhat"])
        
        # exporting the coefficients for the training set
        if(broadcast_coeff_req.value == True):
            results_pd.loc[0,"Coeffs"] = str({'Variable':list(coeffs_df['Variable'].values), 'Value':list(coeffs_df['Value'].values)})
        else:
            results_pd.loc[0,"Coeffs"] = "null"
            
        # to handle erroneous results epsilon is set to 1.
        epsilon = 1

        temp_data1 = pd.DataFrame(index= range(1))
        temp_data2 = pd.DataFrame()
        results_pd_temp = results_pd[~((results_pd['test_flag']==1) & (results_pd['test_flag_agg']==0))]
        for val in [1,0]:
            temp_data = results_pd_temp[results_pd_temp['test_flag_agg']==val]
            y_pred = temp_data['yhat']
            y_true = temp_data['y']

            temp_data1['test_flag_agg'] = val
            # Eval. metrics calculation
            temp_data1['mape'] = np.mean(np.abs(y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))*100  
            temp_data1['wmape'] = np.sum(np.abs(y_true - y_pred)) / np.maximum(np.sum(np.abs(y_true)),epsilon)*100  
            temp_data1['bias'] = np.mean((y_true - y_pred))  
            temp_data1['tracking_signal'] = np.sum((y_true - y_pred)) / np.mean(np.abs(y_true - y_pred))
            temp_data1['mae'] = mean_absolute_error(y_true, y_pred)
            temp_data1['rmse']=np.sqrt(mean_squared_error(y_true, y_pred))
            temp_data2 = pd.concat([temp_data2,temp_data1],ignore_index = True)

        results_pd = pd.merge(results_pd,temp_data2,how='left',on='test_flag_agg')
      
        # To adhere to defined schema
        for x in broadcast_gran:   
            results_pd[x] = results_pd[x].astype(str)

        temp_list = temp_list + list(set(regressors) - set(coeffs_df['Variable'].values))
        if(len(temp_list)>0):
            for var in temp_list:
                results_pd[var] = 0
        results_pd.loc[results_pd['yhat']==0,list(coeffs_df['Variable'])] = 0
        
        # Get the experiment id
        tracking_value = broadcast_tracking.value.copy()
        if(mlflow_tracking_check.value == "Out of Sample" and tracking_value["tracking_needed"] == True):
            if(tracking_value['type']!="Managed"):
                if(tracking_value['tracking_uri'] is not None):
                    mlflow.set_tracking_uri("file:"+tracking_value['tracking_uri'])
                    experiment_id = mlflow.set_experiment(tracking_value["mlflow_experiment_id"])
                    tracking_value['mlflow_experiment_id'] = experiment_id.experiment_id
            #Add MLFlow code here
            with mlflow.start_run(experiment_id = tracking_value['mlflow_experiment_id']):
                mlflow.log_param('algorithm', 'Lasso_cvglmnet')
                mlflow.log_param('result_type', 'out_of_sample')
                for x in broadcast_gran:
                    mlflow.log_param(x, results_pd[x].iloc[0])
                temp_test = results_pd[results_pd['test_flag']==1].reset_index(drop = True)
                for x in ["mape","wmape","bias","tracking_signal","mae","rmse"]:
                    mlflow.log_metric(x, temp_test[x].iloc[0])
		
        results_pd["window"] = window_no
        results_pd['status'] = 'success'
        return results_pd
      
    except Exception as e:
        results_pd = pd.DataFrame(columns = [['ds', 'y', 'yhat','mape','wmape','bias','tracking_signal','mae','rmse']+\
                                              ['status','test_flag','test_flag_agg','window','Coeffs'] + broadcast_granularity.value+\
                                               broadcast_regressors.value + ['Intercept']],index = range(1))
        results_pd[broadcast_granularity.value] = df_data[broadcast_granularity.value].head(1).reset_index(drop = True)
        for x in broadcast_granularity.value:
            results_pd[x] = results_pd[x].astype(str)
        results_pd['status'] = str(e) 
        return results_pd

#### Loading the latest Missing_value_treatment file
##### Please update the reading path with the required data path if "Missing value treatment" was not run

In [0]:
# Reading the latest input file based on timestamp
all_files = [file for file in os.listdir(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment")]
missing_op_files = [file for file in all_files if "Missing_value_treatment_results (" in file]
missing_op_files = [file.replace(".csv","") for file in missing_op_files]
version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in missing_op_files]
max_date = max(version_dates)
max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
req_file_name = [x for x in missing_op_files if max_date in x]
missing_op_file_path = os.path.join(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment",req_file_name[0] + ".csv")
# print(missing_op_file_path)

# Reading the data
df = pd.read_csv(missing_op_file_path)
# print(df.shape)

df.rename(columns = {ds_config:"ds", dv_config:"y"}, inplace = True)
logger.info("Data loaded")

df['ds'] = pd.to_datetime(df['ds'])
df[modeling_granularity_conf] = df[modeling_granularity_conf].astype(str)
gbcp = list(modeling_granularity_conf)

In [0]:
if(app_config["validation"]["agg_metrics_req"]):

    # Creating windows and then calling the modeling function
    test_periods = int(broadcast_test_periods.value)
    window_test_periods = app_config["validation"]["agg_metrics_test_periods"]
    stride = app_config["validation"]["agg_metrics_stride"]

    # Getting the total number of weeks for each time series
    temp_df = df.groupby(modeling_granularity_conf).agg({'ds':'count'}).rename(columns={'ds': '#total_weeks'}).reset_index()
    df = df.merge(temp_df, on = modeling_granularity_conf ,how = "left")

    unique_skuXds = df[modeling_granularity_conf+["#total_weeks"]].drop_duplicates().reset_index(drop = True)

    final_list = []
    gran_len = len(modeling_granularity_conf)
    
    for row1 in range(0,len(unique_skuXds)): 
        Total_weeks = unique_skuXds.loc[row1,'#total_weeks']
        train_interval = int(Total_weeks-test_periods)
        j = 0
        for train_i in range(train_interval,Total_weeks,stride):
            if(train_i+window_test_periods <=Total_weeks):
                test_i = train_i+window_test_periods
                final_list.append([unique_skuXds.iloc[row1,index] for index in range(gran_len)] + [0,train_i,train_i+window_test_periods,j+1])
                j += 1

    # create all windows combination.
    df_windows = pd.DataFrame([tuple(x) for x in final_list],columns =modeling_granularity_conf+['train_index_start','train_index_end','test_index_end','window_no'])
    f_df = df.merge(df_windows,on=modeling_granularity_conf,how="left")
        
    f_df['gran_tempp'] = f_df[gbcp+["window_no"]].astype(str).sum(axis=1)
    unique_pdts = f_df['gran_tempp'].unique()
    new_results = pd.DataFrame()
    for pdt in unique_pdts:
        new_results = pd.concat([new_results,get_forecast_UDF(f_df[f_df['gran_tempp']==pdt])])
            
    new_results.to_csv(algo_path+"/Out_of_sample_results_window_level ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
    logger.info("Completed Backtesting")
    
    # Reading the latest Out_of_sample_results_window_level file based on timestamp
    all_files = [file for file in os.listdir(algo_path)]
    backtesting_files = [file for file in all_files if "Out_of_sample_results_window_level (" in file]
    backtesting_files = [file.replace(".csv","") for file in backtesting_files]
    version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in backtesting_files]
    max_date = max(version_dates)
    max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
    req_file_name = [x for x in backtesting_files if max_date in x]
    backtesting_results_file_path = os.path.join(algo_path,req_file_name[0] + ".csv")
    req_cols_coeffs = ['window','Coeffs']
    print(backtesting_results_file_path)

    # Reading the results of backtesting
    df = pd.read_csv(backtesting_results_file_path)
    df = df[df["status"] == "success"]
    
    df[modeling_granularity_conf] = df[modeling_granularity_conf].astype(str)
    df['ds'] = pd.to_datetime(df['ds'])

    # performance metrics
    per_met = ['status',"test_flag_agg","window","mape","wmape","bias","tracking_signal","mae","rmse"]
    df_metrics = df[gbcp + per_met].drop_duplicates()
    df_metrics1 = df_metrics.groupby(gbcp + ['test_flag_agg','status'])[["mape","wmape","bias","tracking_signal","mae","rmse"]].mean().reset_index()

    # Remaining columns
    rem_cols = list(set(df.columns) - set(per_met)) + ['test_flag_agg']
    rem_cols.remove('Coeffs')
    dot_cols = [col for col in df.columns if "." in col] #to handle "."s
    for col in dot_cols:
        df.rename(columns = {col:col.replace(".","dot")}, inplace = True)
        rem_cols[rem_cols.index(col)] = col.replace(".","dot")
    rem_df = df[rem_cols]
    
    # Removing the training dates which falls in the test period
    rem_df = rem_df[~((rem_df['test_flag']==1) & (rem_df['test_flag_agg']==0))]
    group_cols = gbcp + ['ds','test_flag','test_flag_agg']
    agg_cols = list(set(rem_cols) - set(group_cols))
    exprs = {x: "mean" for x in agg_cols}
    rem_df1 = rem_df.groupby(group_cols).agg(exprs).reset_index()
    temp_cols = [col[:-1] if 'avg(' in col else col for col in rem_df1.columns ]
    temp_cols = [col.replace('avg(','') for col in temp_cols]
    rem_df1.columns = temp_cols

    for col in dot_cols:
        rem_df1.rename(columns = {col.replace(".","dot"):col.replace("dot",".")}, inplace = True)
                            
    # combining all the data
    df_forecast = rem_df1.merge(df_metrics1, on = gbcp + ['test_flag_agg'], how='left')
    
else:    
    df['gran_tempp'] = df[gbcp].astype(str).sum(axis=1)
    unique_pdts = df['gran_tempp'].unique()
    df_forecast = pd.DataFrame()
    for pdt in unique_pdts:
        df_forecast = pd.concat([df_forecast,get_forecast_UDF(df[df['gran_tempp']==pdt])])
            
del(df_forecast['test_flag_agg'])
df_forecast['algorithm'] = 'Lasso_cvglmnet'

<command-3460216454770823>:237: DeprecationWarning: scipy.empty is deprecated and will be removed in SciPy 2.0.0, use numpy.empty instead
 nz = glmnetPredict(glmfit, scipy.empty([0]), scipy.empty([0]), 'nonzero')
<command-3460216454770823>:246: DeprecationWarning: scipy.sum is deprecated and will be removed in SciPy 2.0.0, use numpy.sum instead
 nz = scipy.transpose(scipy.sum(nz, axis = 0))
<command-3460216454770823>:246: DeprecationWarning: scipy.transpose is deprecated and will be removed in SciPy 2.0.0, use numpy.transpose instead
 nz = scipy.transpose(scipy.sum(nz, axis = 0))
<command-3460216454770823>:249: DeprecationWarning: scipy.arange is deprecated and will be removed in SciPy 2.0.0, use numpy.arange instead
 ma = scipy.tile(scipy.arange(nfolds), [1, int(scipy.floor(nobs/nfolds))])
<command-3460216454770823>:249: DeprecationWarning: scipy.floor is deprecated and will be removed in SciPy 2.0.0, use numpy.floor instead
 ma = scipy.tile(scipy.arange(nfolds), [1, int(scipy.floor(nobs/nfolds))])
<command-3460216454770823>:249: DeprecationWarning: scipy.tile is deprecated and will be removed in SciPy 2.0.0, use numpy.tile instead
 ma = scipy.tile(scipy.arange(nfolds), [1, int(scipy.floor(nobs/nfolds))])
<command-3460216454770823>:250: DeprecationWarning: scipy.mod is deprecated and will be removed in SciPy 2.0.0, use numpy.mod instead
 mb = scipy.arange(scipy.mod(nobs, nfolds))
<command-3460216454770823>:250: DeprecationWarning: scipy.arange is deprecated and will be removed in SciPy 2.0.0, use numpy.arange instead
 mb = scipy.arange(scipy.mod(nobs, nfolds))
<command-3460216454770823>:251: DeprecationWarning: scipy.reshape is deprecated and will be removed in SciPy 2.0.0, use numpy.reshape instead
 mb = scipy.reshape(mb, [1, mb.size])
<command-3460216454770823>:252: DeprecationWarning: scipy.append is deprecated and will be removed in SciPy 2.0.0, use numpy.append instead
 population = scipy.append(ma, mb, axis = 1)
<command-3460216454770823>:256: DeprecationWarning: scipy.reshape is deprecated and will be removed in SciPy 2.0.0, use numpy.reshape instead
 foldid = scipy.reshape(foldid, [foldid.size,])
<command-3460216454770823>:264: DeprecationWarning: scipy.reshape is deprecated and will be removed in SciPy 2.0.0, use numpy.reshape instead
 foldid = scipy.reshape(foldid, [foldid.size, ])
<command-3460216454770823>:306: DeprecationWarning: scipy.transpose is deprecated and will be removed in SciPy 2.0.0, use numpy.transpose instead
 CVerr['cvm'] = scipy.transpose(cvm)
<command-3460216454770823>:307: DeprecationWarning: scipy.transpose is deprecated and will be removed in SciPy 2.0.0, use numpy.transpose instead
 CVerr['cvsd'] = scipy.transpose(cvsd)
<command-3460216454770823>:308: DeprecationWarning: scipy.transpose is deprecated and will be removed in SciPy 2.0.0, use numpy.transpose instead
 CVerr['cvup'] = scipy.transpose(cvm + cvsd)
<command-3460216454770823>:309: DeprecationWarning: scipy.transpose is deprecated and will be removed in SciPy 2.0.0, use numpy.transpose instead
 CVerr['cvlo'] = scipy.transpose(cvm - cvsd)
<command-3460216454770823>:318: DeprecationWarning: scipy.amin is deprecated and will be removed in SciPy 2.0.0, use numpy.amin instead
 CVerr['lambda_min'] = scipy.amax(options['lambdau'][cvm <= scipy.amin(cvm)]).reshape([1])
<command-3460216454770823>:318: DeprecationWarning: scipy.amax is deprecated and will be removed in SciPy 2.0.0, use numpy.amax instead
 CVerr['lambda_min'] = scipy.amax(options['lambdau'][cvm <= scipy.amin(cvm)]).reshape([1])
<command-3460216454770823>:321: DeprecationWarning: scipy.amax is deprecated and will be removed in SciPy 2.0.0, use numpy.amax instead
 CVerr['lambda_1se'] = scipy.amax(options['lambdau'][cvm <= semin]).reshape([1])
<command-3460216454770825>:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [0]:
df_forecast.to_csv(algo_path+"/Out_of_sample_evaluation_results ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
logger.info("Exported Out of sample evaluation results")

In [0]:
# Getting the coefficients at granularity level
if(app_config['Algorithms']['Lasso_cvglmnet']['coefficients_required']):
    if(app_config["validation"]["agg_metrics_req"] == False):
        # Reading the latest Out_of_sample_results_window_level file based on timestamp
        all_files = [file for file in os.listdir(algo_path)]
        backtesting_files = [file for file in all_files if "Out_of_sample_evaluation_results (" in file]
        backtesting_files = [file.replace(".csv","") for file in backtesting_files]
        version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in backtesting_files]
        max_date = max(version_dates)
        max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
        req_file_name = [x for x in backtesting_files if max_date in x]
        backtesting_results_file_path = os.path.join(algo_path,req_file_name[0] + ".csv")
        print(backtesting_results_file_path)
        req_cols_coeffs = ['Coeffs']

    out_results = pd.read_csv(backtesting_results_file_path)
    out_results = out_results[out_results["status"] == "success"]
    out_results = out_results[gbcp+req_cols_coeffs].drop_duplicates()
    out_results = out_results[~pd.isna(out_results['Coeffs'])]
    out_results.to_csv(algo_path+"/Coefficients ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
    logger.info("Exported Lasso_cvglmnet Coefficient results")

### Predicting future timeperiods
The following code assumes that the X-variables for the required future time periods are available for each modeling granularity

Uncomment the below cells if wants to predict the future, update the df respectively such that it contains entire historical data as well as idvs data for the required future forecast time periods

In [0]:
# broadcast_test_periods =  broadcast_variable_conf(4) # Provide the no. of timeperiods to forecast in the future

In [0]:
## Reading the latest input file based on timestamp
# all_files = [file for file in os.listdir(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment")]
# missing_op_files = [file for file in all_files if "Missing_value_treatment_results (" in file]
# missing_op_files = [file.replace(".csv","") for file in missing_op_files]
# version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in missing_op_files]
# max_date = max(version_dates)
# max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
# req_file_name = [x for x in missing_op_files if max_date in x]
# missing_op_file_path = os.path.join(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment",req_file_name[0]+'.csv')
## print(missing_op_file_path)

## Reading the data
# df = pd.read_csv(missing_op_file_path)
## print(df.shape)

# df.rename(columns = {ds_config:"ds", dv_config:"y"}, inplace = True)
# df['ds'] = pd.to_datetime(df['ds'])
# df[modeling_granularity_conf] = df[modeling_granularity_conf].astype(str)

# # Broadcasting again with the "Future forecast" value since we won't be tracking the future forecast results
# mlflow_tracking_check = broadcast_required_info("Future forecast")
# logger.info("Data which contains the future forecast periods is loaded")

# gbcp = list(modeling_granularity_conf)

In [0]:
# df['gran_tempp'] = df[gbcp].astype(str).sum(axis=1)
# unique_pdts = df['gran_tempp'].unique()
# df_forecast = pd.DataFrame()
# for pdt in unique_pdts:
#     df_forecast = pd.concat([df_forecast,get_forecast_UDF(df[df['gran_tempp']==pdt])])
            
# del(df_forecast['test_flag_agg'])
# df_forecast['algorithm'] = 'Lasso_cvglmnet'

In [0]:
# df_forecast.to_csv(algo_path + "/Future_forecast_results ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
# logger.info("Exported future forecast results")

In [0]:
# Exporting config file
config_file_name = "config_for_exp_id_"+str(broadcast_tracking.value['mlflow_experiment_id']) + " (" +datetime.today().strftime('%Y-%m-%d-%H-%M-%S-%f')[:-3]+").yml"
config_path1 = os.path.join(config_path,config_file_name)
with open(config_path1, 'w') as file:
    yaml.dump(temp_config, file, default_flow_style=False,sort_keys=False)

In [0]:
# Move from tmp directory to req. location in datalake
import platform
plat_sys = platform.system()

if(plat_sys!='Windows'):
    log_file = log_file.replace(' (', '\ \(').replace(')','\)')
    os.system('mv /tmp/{0} {1}'.format(log_file,logs_path))